# 01. Load the packages

In [14]:

import simserialED.core as simserialED
import h5py
from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental
from diffsims.generators.simulation_generator import SimulationGenerator
from diffpy.structure import Atom, Lattice, Structure
from ase.io import read
from pymatgen.io.ase import AseAtomsAdaptor

print("Imports succeeded!")


Imports succeeded!


In [13]:

# Read the PDB file into an ASE Atoms object
atoms = read("/Users/xiaodong/Desktop/UOXs/UOX.pdb")

# Convert the ASE Atoms object to a pymatgen Structure
structure = AseAtomsAdaptor.get_structure(atoms)

# Extract lattice parameters from pymatgen structure
lat = structure.lattice
a = lat.a
b = lat.b
c = lat.c
alpha = lat.alpha
beta  = lat.beta
gamma = lat.gamma

# Create a diffpy Lattice object using these parameters
lattice_diffpy = Lattice(a, b, c, alpha, beta, gamma)

# Build a list of diffpy Atom objects
atoms_diffpy = []
# We convert fractional coordinates to Cartesian coordinates
for site in structure.sites:
    # Convert fractional coordinates to Cartesian
    # You can also directly use site.coords if available (which are in Cartesian)
    cart_coords = lat.get_cartesian_coords(site.frac_coords)
    # Create the Atom; note that site.species_string gives a string for the species
    atoms_diffpy.append(Atom(site.species_string, tuple(cart_coords)))

# Create a diffpy Structure using the atoms and lattice
s_diffpy = Structure(atoms_diffpy, lattice_diffpy)

# Finally, create the Phase object.
# Here, we specify a space group (for example, 1). Adjust this if you know the correct space group.
p = Phase(space_group=23, structure=s_diffpy)

# Suppose you already created your Phase p with space_group=23 (I222)
# print("Space group:", p.space_group)

# p.point_group = "222"

# print("Assigned point group:", p.point_group)
# Print or work with your Phase object
# print(p.structure)

oris = get_sample_reduced_fundamental(
    resolution=20, # degrees, angular distance between simulations
    point_group=p.point_group
) 
# perform simulations
# Initialize simulation generator. This takes in parameters like scattering table, shape factor model ect.
gen = SimulationGenerator(
    accelerating_voltage=300,
)

print(p)


<name: . space group: I222. point group: 222. proper point group: 222. color: tab:blue>


In [9]:
from pprint import pprint

print("Inspecting Phase object:\n")

print("---------- Approach 1: vars(p) ----------")
try:
    pprint(vars(p))
except TypeError:
    print("vars(p) failed; the object might use __slots__ or not be a typical Python object.\n")

print("\n---------- Approach 2: dir/pprint ----------")
for attr_name in dir(p):
    if attr_name.startswith("__"):
        continue
    try:
        attr_value = getattr(p, attr_name)
        print(f"{attr_name}: {attr_value}")
    except AttributeError:
        print(f"{attr_name}: <unreadable attribute>")

if hasattr(p, "info") and callable(p.info):
    print("\n---------- Approach 3: p.info() ----------")
    p.info()


Inspecting Phase object:

---------- Approach 1: vars(p) ----------
{'_color': 'tab:blue',
 '_diffpy_lattice': array([[ 80.58,   0.  ,   0.  ],
       [  0.  ,  94.49,   0.  ],
       [  0.  ,   0.  , 103.89]]),
 '_point_group': None,
 '_space_group': SpaceGroup #23 (I222, Orthorhombic). Symmetry matrices: 8, point sym. matr.: 4,
 '_structure': [N    61.343000 57.912000 20.514000 1.0000,
                C    60.060000 57.287000 20.916000 1.0000,
                C    58.926000 58.260000 20.646000 1.0000,
                O    59.138000 59.347000 20.106000 1.0000,
                C    60.094000 56.898000 22.394000 1.0000,
                O    61.079000 55.905000 22.626000 1.0000,
                H    61.771000 57.375000 19.948000 1.0000,
                H    61.182000 58.692000 20.117000 1.0000,
                H    61.848000 58.045000 21.235000 1.0000,
                H    59.907000 56.473000 20.410000 1.0000,
                H    60.305000 57.683000 22.923000 1.0000,
                H  

In [12]:

sims = gen.calculate_diffraction2d(
    phase=p,
    rotation=oris,
    reciprocal_radius=1,
    with_direct_beam=False,
    max_excitation_error=0.03,
    shape_factor_width=None,
    debye_waller_factors=None,
    show_progressbar=True
)


KeyboardInterrupt: 

In [ ]:

# Generate simulations for our structure and chosen orientations
sims = gen.calculate_diffraction2d(
    phase=p,
    rotation=oris,
    reciprocal_radius=1,    # 1/Å, NOT Å. However, 1 1/Å corresponds to 1Å, so in this case it's fine
    with_direct_beam=True, # true/false if we want to include the direct beam
    max_excitation_error = 0.0003,
    shape_factor_width = None,
    debye_waller_factors = None,
    show_progressbar = True
)
# Note that these simulations are a set of diffraction vectors, with x/y coordinates and intensity, for each orientation. We will later convert this to a diffraction pattern image

In [ ]:

# Define structure

# Read the PDB file into an ASE Atoms object
atoms = read("/Users/xiaodong/Desktop/UOXs/UOX.pdb")

# Convert the ASE Atoms object to a pymatgen Structure
structure = AseAtomsAdaptor.get_structure(atoms)

# Extract lattice parameters from pymatgen structure
lat = structure.lattice
a = lat.a
b = lat.b
c = lat.c
alpha = lat.alpha
beta  = lat.beta
gamma = lat.gamma

# Create a diffpy Lattice object using these parameters
lattice_diffpy = Lattice(a, b, c, alpha, beta, gamma)

# Build a list of diffpy Atom objects
atoms_diffpy = []
# We convert fractional coordinates to Cartesian coordinates
for site in structure.sites:
    # Convert fractional coordinates to Cartesian
    # You can also directly use site.coords if available (which are in Cartesian)
    cart_coords = lat.get_cartesian_coords(site.frac_coords)
    # Create the Atom; note that site.species_string gives a string for the species
    atoms_diffpy.append(Atom(site.species_string, tuple(cart_coords)))

# Create a diffpy Structure using the atoms and lattice
s_diffpy = Structure(atoms_diffpy, lattice_diffpy)

# Finally, create the Phase object.
# Here, we specify a space group (for example, 1). Adjust this if you know the correct space group.
p = Phase(space_group=1, structure=s_diffpy)

# Print or work with your Phase object
# print(p.structure)

oris = get_sample_reduced_fundamental(
    resolution=20, # degrees, angular distance between simulations
    point_group=p.point_group
) 
# perform simulations
# Initialize simulation generator. This takes in parameters like scattering table, shape factor model ect.
gen = SimulationGenerator(
    accelerating_voltage=300,
)
# Generate simulations for our structure and chosen orientations
sims = gen.calculate_diffraction2d(
    phase=p,
    rotation=oris,
    reciprocal_radius=1,    # 1/Å, NOT Å. However, 1 1/Å corresponds to 1Å, so in this case it's fine
    with_direct_beam=True, # true/false if we want to include the direct beam
)
# Note that these simulations are a set of diffraction vectors, with x/y coordinates and intensity, for each orientation. We will later convert this to a diffraction pattern image

# 02. Load the data

In [ ]:
filename = r"C:\Users\vife5188\Documents\data\UOX_His_MUA_450nm_spot4_ON_20240310_0931_run_2024-03-12_13-06-30.h5"
data = h5py.File(filename)

# Inspect the file by printing the keys
print(simserialED.data.serialize_hdf5(data))

# 03. Perform simulations

## The easy way

In [ ]:
sims = simserialED.simulation.get_simulations_with_params(
    # Unit cell
    a = 20,         # Å
    b = 10,         # Å
    c = 15,         # Å
    alpha=90,       # deg
    beta=90,        # deg
    gamma=90,       # deg
    
    spacegroup=19,  # int, not inferred from unit cell
    kv=200,         # kV, acelleration voltage, to determine radius of Ewald's sphere
    angres=1,       # deg, angular resolution of simulations, i.e. the rotation angle in degrees between neighbouring simulations. Lower number = more simulations
)

In [ ]:
# or even easier:
sims = simserialED.simulation.get_simulations()
# This is lysozyme (pdb entry 7SKW), but with only a single carbon atom at [0 0 0] 

## The complex way

In [ ]:
# import libraries
from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental
from diffsims.generators.simulation_generator import SimulationGenerator

In [ ]:
# Define structure
# can be read from cif:
p = Phase.from_cif("data/lysozyme.cif") 

# or defined manually:
from diffpy.structure import Atom, Lattice, Structure
l = Lattice(10, 20, 30, 90, 100, 110)
atoms = [
    Atom("C", (0, 0, 0)),
    Atom("Fe", (0, 0.5, 0.5)),
    # ect.
]
s = Structure(atoms, l)
p = Phase(space_group=1, structure=s)

In [ ]:
# Choose orientations to simulate
# Can sample the symmetry reduced zone of a given point group:
oris = get_sample_reduced_fundamental(
    resolution=1, # degrees, angular distance between simulations
    point_group=p.point_group
) # This effectively evenly samples the two first Euler angles, ignoring in-plane rotation 
# (as we usually don't care about that when simulating diffraction patterns, 
# and it's much easier to just rotate the generated simulations than perform more simulations)

# Or define the orientations manually, from e.g. Euler angles:
from orix.quaternion import Rotation
oris = Rotation.from_euler([
        # phi1, Psi, phi2
        [10, 30, 0],
        [11, 30, 0],
        [10, 31, 0],
        [11, 31, 0],
    ], 
    degrees=True,
)
# More options are available from Rotation's class methods (e.g. rotation matrix).
# As a sidenote, the underlying data of orientations are stored as quaternions, so if they are initialized without any from-method they need 4 numbers per orientation

In [ ]:
# perform simulations
# Initialize simulation generator. This takes in parameters like scattering table, shape factor model ect.
gen = SimulationGenerator(
    accelerating_voltage=200,
)
# Generate simulations for our structure and chosen orientations
sims = gen.calculate_diffraction2d(
    phase=p,
    rotation=oris,
    reciprocal_radius=1,    # 1/Å, NOT Å. However, 1 1/Å corresponds to 1Å, so in this case it's fine
    with_direct_beam=False, # true/false if we want to include the direct beam
)
# Note that these simulations are a set of diffraction vectors, with x/y coordinates and intensity, for each orientation. We will later convert this to a diffraction pattern image

In [ ]:
# Optionally, plot the simulations.
# This includes a slider to interactively view the different orientations
sims.plot()

# 04. Add simulations to dataset

In [ ]:
import random

# You might need/want different keys here, depending on your dataset layout
images = data["entry"]["data"]["images"]

# These are for storing the simulations
sim_images = data["entry"]["data"].require_dataset("simulation_images", shape=images.shape, dtype=images.dtype)
euler_angles = data["entry"]["data"].require_dataset("simulation_euler_angles", shape=(images.shape[0], 3), dtype=float)
# TODO store coordinates too
# coordinates = data["entry"]["data"].require_dataset("simulation_coordinates", shape=(images.shape[0], ), dtype=float)

shape = images.shape[-2:]
calibration = 1 / 3.9 / 183 # 1/Å per pixel. 
intensity_scale = 1000 # All simulations have intensities between 0 and 1. These are multiplied by intensity_scale for the images.

from tqdm import tqdm # for a nice progressbar
for i in tqdm(range(images.shape[0])):
    # Choose a random in-plane rotation
    in_plane = random.uniform(0, 360)

    # Choose a random simulation
    ind = random.randint(0, sims.current_size - 1)

    # Generate simulation image
    img = sims.irot[ind].get_diffraction_pattern(
        shape=shape,
        direct_beam_position=shape // 2, # pixel coordinates of direct beam
        sigma=1,                        # sigma in gaussian blur for spot size. Otherwise there is only a single pixel per reflection
        in_plane_angle=in_plane,
        calibration=calibration,
    )

    # Scale image intensity
    img = (img * intensity_scale).astype(images.dtype)

    # Add simulation to the data
    images[i] += img

    # Store the euler angles, to recreate the orientation corresponding to the simulation
    euler_angles_i = sims.rotations[ind].to_euler(degrees=True).squeeze()
    euler_angles_i[0] = in_plane
    euler_angles[i] = euler_angles_i

    # Store the simulation
    sim_images[i] = img

# 05. Inspect results

In [ ]:
# Interactive plot
from matplotlib import pyplot as plt
plt.figure()
plt.subplots_adjust(bottom=0.2)
plt.subplot(1, 2, 1)
sim = plt.imshow(sim_images[0])
text = plt.text(0, 0, "tmp")
plt.axis("off")
plt.subplot(1, 2, 2)
data = plt.imshow(images[0])
plt.axis("off")

from orix.quaternion import Orientation
from orix.vector import Miller
z = Miller(uvw=[0, 0, 1], phase=p) # Use phase from before


from matplotlib.widgets import Slider

s = Slider(plt.axes((0.1, 0.1, 0.8, 0.05)), "ind", 0, sim_images.shape[0])

def on_changed(val):
    ind = int(s.val)
    sim.set_data(sim_images[ind])
    data.set_data(images[ind])
    o = Orientation.from_euler(euler_angles[ind], degrees=True, symmetry=p.point_group)
    text.set_text(str((o * z).round()))

s.on_changed(on_changed)